# Calculations of the Effect Size (ES) for each microarray study 

###  Using Hedges' g value, an adjusted Cohen's d  value

$$  {Enrichment} = \bar{X_2}-\bar{X_1}$$

Let Group 1 be 6hSleeping whole brain AKR/J strain (A) Expression values and Group 2 be 6hSD whole brain AKR/J strain (A) Expression values 

(S mean - SD mean) **(Logged values, so minus gives ratio)** 

$$  {Pooled\ Standard\  Deviation} = \sqrt\frac{(n_1-1)S_1^2 +(n_2-1)S_2^2}{(n_2 +n_2) -2}  $$  

$$  {Cohen's\ d\ value} = \frac{Enrichment}{Pooled\ Standard\ Deviation} $$

$$  {Correction\ Factor (J\ Factor)} = 1- \frac{3}{4df-1} $$

$$  {Hedges'\ g\ value} = Cohen's\ d\ \text{x}\ J\ $$

$$  {Variance\ in\ d (V_d)} = \frac{n_1- +n_2}{n_1 n_2} + \frac{d^2}{2(n_1 +n_2)}  $$

$$  {Variance\ in\ g (V_g)} = J^2\  \text{x}\ V_d  $$

$$  {Standard\ Error\ in\ g (SE_g)} = \sqrt{V_g}  $$

## Setup working environment and import data

In [1]:
import pandas as pd # Dataframes and file IO
import numpy as np # numerical calculations
%cd /Users/Ella1/Desktop/data sets 430AV2


/Users/Ella1/Desktop/data sets 430AV2


In [2]:
prefix = '430AV2_ZT6_AK_'   # define a prefix to add to column names (making indexing easier later)

In [3]:
# import the data file to a data frame 'df'
df=pd.read_table('DATASET-GSE9442.txt', delimiter='\t',  index_col=0) #,nrows=500)  
df.shape

(45101, 87)

In [4]:
# remove probes that are know to cross-hybridise to more than one target
df =df[~df.index.str.contains('_x_|_s_')]    #   important reverse selector ~ 
df.shape

(40569, 87)

## Look at column names and then setup filters for grouping columns into S and SD groups

In [5]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites', 'Select Cellular Compartments',
       'Select Protein Classes', 'GSM239868_AK_S_ZT6.CEL',
       'GSM239869_AK_S_ZT6.CEL', 'GSM239870_AK_S_ZT6.CEL', 'avg-AK_S_ZT6',
       'log_fold-AK_S_ZT6_vs_AK_SD_ZT6', 'fold-AK_S_ZT6_vs_AK_SD_ZT6',
       'rawp-AK_S_ZT6_vs_AK_SD_ZT6', 'adjp-AK_S_ZT6_vs_AK_SD_ZT6',
       'GSM239871_AK_S_ZT12.CEL', 'GSM239872_AK_S_ZT12.CEL',
       'GSM239873_AK_S_ZT12.CEL', 'avg-AK_S_ZT12',
       'log_fold-AK_S_ZT12_vs_AK_SD_ZT12', 'fold-AK_S_ZT12_vs_AK_SD_ZT12',
       'rawp-AK_S_ZT12_vs_AK_SD_ZT12', 'adjp-AK_S_ZT12_vs_AK_SD_ZT12',
       'GSM239880_AK_SD_ZT6.CEL', 'GSM239881_AK_SD_ZT6.CEL',
       'GSM239882_AK_SD_ZT6.CEL', 'avg-AK_SD_ZT6', 'GSM239883_AK_SD_ZT12.CEL',
       'GSM239884_AK_SD_ZT12.CEL', 'GSM239885_AK_SD_ZT12.CEL',
       'avg-AK_SD_ZT12', 'GSM239891_B6_S_ZT6.CEL',
  

In [6]:
# define regular expressions for sleep (S) and sleep dep (SD) filters 
s_filt ='AK_S_ZT6.CEL'
sd_filt ='AK_SD_ZT6.CEL'

In [7]:
df_s=df.filter(regex= s_filt)
df_s.head()

,GSM239868_AK_S_ZT6.CEL,GSM239869_AK_S_ZT6.CEL,GSM239870_AK_S_ZT6.CEL
Probesets,,,
1427138_at,6.35703,6.18535,6.36848
1425600_a_at,7.28398,7.57596,7.18557
1457168_at,4.67183,4.79144,4.88085
1450135_at,4.90978,5.10407,5.13015
1424014_at,8.29628,8.24320,8.32830


In [8]:
df_sd=df.filter(regex= sd_filt)
df_sd.head()

,GSM239880_AK_SD_ZT6.CEL,GSM239881_AK_SD_ZT6.CEL,GSM239882_AK_SD_ZT6.CEL
Probesets,,,
1427138_at,6.09301,6.31847,6.11162
1425600_a_at,7.44672,7.48677,7.54731
1457168_at,4.68634,4.60157,4.48238
1450135_at,5.22263,5.28250,4.99002
1424014_at,8.11648,8.33775,8.18348


## Calculations 

In [9]:
# Enrichment

df[prefix+'Enrich'] = df.filter(regex=sd_filt).mean(axis=1) - df.filter(regex=s_filt).mean(axis=1)

In [10]:
df[prefix+'Enrich'].head()

Probesets
1427138_at     -0.129253
1425600_a_at    0.145097
1457168_at     -0.191277
1450135_at      0.117050
1424014_at     -0.076690
Name: 430AV2_ZT6_AK_Enrich, dtype: float64

In [11]:
# Calculating Pooled StDev
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

StdevS = (Scount-1) * df.filter(regex=s_filt).var(axis=1)
StdevSD = (SDcount-1) * df.filter(regex=sd_filt).var(axis=1)

df[prefix+'poolStDev'] = np.sqrt((StdevS+StdevSD)/(Scount+ SDcount-2))

In [12]:
# Calculating Cohen's d
df[prefix+'Cohens_d'] = df[prefix+'Enrich'] / df[prefix+'poolStDev']

In [13]:
#df[prefix+'poolStDev'].head()
df[prefix+'Cohens_d'] .head()

Probesets
1427138_at     -1.129625
1425600_a_at    0.980606
1457168_at     -1.844965
1450135_at      0.845051
1424014_at     -0.893855
Name: 430AV2_ZT6_AK_Cohens_d, dtype: float64

In [14]:
# Calculating J value (Correction factor)

df[prefix+'J'] = 1-(3/(4*(Scount+SDcount-1)))                              


In [15]:
# Calculating Hedge's g

df[prefix+'Hedges_g'] = df[prefix+'Cohens_d'] * df[prefix+'J']

In [16]:
#df[prefix+'J'].head()
df[prefix+'Hedges_g'] .head()

Probesets
1427138_at     -0.960181
1425600_a_at    0.833515
1457168_at     -1.568220
1450135_at      0.718293
1424014_at     -0.759776
Name: 430AV2_ZT6_AK_Hedges_g, dtype: float64

In [17]:
# Calculating Var_d
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

Ftop1 = Scount + SDcount
Ftop2 = Scount * SDcount
Fbottom1 = np.square(df[prefix+'Cohens_d']) 
Fbottom2 =  2*(Scount + SDcount)


df[prefix+'Var_d'] = (Ftop1/Ftop2) + (Fbottom1 /Fbottom2)

In [18]:
#check output
df[prefix+'Var_d'].head()

Probesets
1427138_at      0.773004
1425600_a_at    0.746799
1457168_at      0.950325
1450135_at      0.726176
1424014_at      0.733248
Name: 430AV2_ZT6_AK_Var_d, dtype: float64

In [19]:
df[prefix+'Var_g'] = df[prefix+'Var_d'] * np.square(df[prefix+'J'])

In [20]:
# Calculating SEg
df[prefix+'SEg'] = np.sqrt(df[prefix+'Var_g'])

In [21]:
df.sort_values(by= '430AV2_ZT6_AK_Hedges_g', ascending=False, inplace=True)
df

,Symbol,Definition,Ensembl_id,Entrez_id,Unigene_id,GO-Process,GO-Function,GO-Component,Pathway_info,Putative microRNA binding sites,...,ANOVA-adjp,largest fold,430AV2_ZT6_AK_Enrich,430AV2_ZT6_AK_poolStDev,430AV2_ZT6_AK_Cohens_d,430AV2_ZT6_AK_J,430AV2_ZT6_AK_Hedges_g,430AV2_ZT6_AK_Var_d,430AV2_ZT6_AK_Var_g,430AV2_ZT6_AK_SEg
Probesets,,,,,,,,,,,,,,,,,,,,,
1452754_at,Creld2,cysteine-rich with EGF-like domains 2,ENSMUSG00000023272,76737,NaN,NaN,calcium ion binding,extracellular region // endoplasmic reticulum,NaN,"mmu-miR-186(miRanda), mmu-miR-340-5p(TargetSca...",...,8.608892e-11,1.309973,0.596820,0.039374,15.157683,0.85,12.884031,19.812946,14.314854,3.783498
1419287_at,Tmem208,transmembrane protein 208,ENSMUSG00000014856,66320,NaN,NaN,NaN,integral to membrane // membrane,NaN,"mmu-miR-103(RNAhybrid|TargetScan|miRanda), mmu...",...,1.601119e-05,0.385263,0.179243,0.011881,15.086568,0.85,12.823583,19.633712,14.185357,3.766345
1422480_at,Snx3,sorting nexin 3,ENSMUSG00000019804,54198,NaN,cell communication // transport // protein tra...,protein binding // phosphoinositide binding,NaN,NaN,"mmu-let-7a(RNAhybrid|miRanda), mmu-miR-1(RNAhy...",...,3.955735e-05,0.245747,0.177883,0.013054,13.626236,0.85,11.582301,16.139526,11.660808,3.414792
1417149_at,P4ha2,"procollagen-proline, 2-oxoglutarate 4-dioxygen...",ENSMUSG00000018906,18452,NaN,oxidation reduction // peptidyl-proline hydrox...,"oxidoreductase activity, acting on single dono...",endoplasmic reticulum lumen // endoplasmic ret...,NaN,"mmu-let-7a(RNAhybrid|miRanda|pictar), mmu-let-...",...,1.030603e-06,0.630647,0.302247,0.023763,12.718983,0.85,10.811136,14.147712,10.221722,3.197143
1451165_at,Lmbr1l,limb region 1 like,ENSMUSG00000022999,74775,NaN,endocytosis,receptor activity,plasma membrane // integral to membrane // mem...,NaN,"mmu-let-7a(RNAhybrid|miRanda), mmu-let-7b(RNAh...",...,6.417368e-01,0.264223,0.109650,0.010044,10.917312,0.85,9.279715,10.598975,7.657759,2.767266
1426221_at,Vwa5a,von Willebrand factor A domain containing 5A,ENSMUSG00000023186,67776,NaN,negative regulation of cell cycle // cell cycle,NaN,NaN,NaN,"mmu-miR-101a(miRanda), mmu-miR-1192(miRanda), ...",...,8.591613e-05,0.614617,0.297570,0.029735,10.007486,0.85,8.506363,9.012481,6.511518,2.551768
1439947_at,Cyp11a1,NaN,ENSMUSG00000097604,13070,NaN,response to cAMP // steroid metabolic process ...,cholesterol binding // oxidoreductase activity...,mitochondrion // mitochondrial membrane // mit...,GenMAPP-Glucocorticoid_Mineralocorticoid_Metab...,NaN,...,2.897078e-01,0.404670,0.317437,0.031771,9.991367,0.85,8.492662,8.985618,6.492109,2.547962
1443992_at,Nipbl,Nipped-B homolog (Drosophila) [Source:MGI Symb...,ENSMUSG00000022141,71175,NaN,cell cycle,binding,nucleus,NaN,"mmu-miR-1(miRanda), mmu-miR-100(TargetScan|miR...",...,5.025976e-02,0.313433,0.301703,0.030630,9.849828,0.85,8.372354,8.751592,6.323026,2.514563
1425187_at,Sel1l,sel-1 suppressor of lin-12-like (C. elegans),ENSMUSG00000020964,20338,NaN,Notch signaling pathway,binding,endoplasmic reticulum membrane // endoplasmic ...,NaN,"mmu-let-7a(miRanda), mmu-let-7b(miRanda), mmu-...",...,1.692494e-01,0.301043,0.257497,0.026361,9.768106,0.85,8.302890,8.617991,6.226498,2.495295


In [22]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites', 'Select Cellular Compartments',
       'Select Protein Classes', 'GSM239868_AK_S_ZT6.CEL',
       'GSM239869_AK_S_ZT6.CEL', 'GSM239870_AK_S_ZT6.CEL', 'avg-AK_S_ZT6',
       'log_fold-AK_S_ZT6_vs_AK_SD_ZT6', 'fold-AK_S_ZT6_vs_AK_SD_ZT6',
       'rawp-AK_S_ZT6_vs_AK_SD_ZT6', 'adjp-AK_S_ZT6_vs_AK_SD_ZT6',
       'GSM239871_AK_S_ZT12.CEL', 'GSM239872_AK_S_ZT12.CEL',
       'GSM239873_AK_S_ZT12.CEL', 'avg-AK_S_ZT12',
       'log_fold-AK_S_ZT12_vs_AK_SD_ZT12', 'fold-AK_S_ZT12_vs_AK_SD_ZT12',
       'rawp-AK_S_ZT12_vs_AK_SD_ZT12', 'adjp-AK_S_ZT12_vs_AK_SD_ZT12',
       'GSM239880_AK_SD_ZT6.CEL', 'GSM239881_AK_SD_ZT6.CEL',
       'GSM239882_AK_SD_ZT6.CEL', 'avg-AK_SD_ZT6', 'GSM239883_AK_SD_ZT12.CEL',
       'GSM239884_AK_SD_ZT12.CEL', 'GSM239885_AK_SD_ZT12.CEL',
       'avg-AK_SD_ZT12', 'GSM239891_B6_S_ZT6.CEL',
  

### Import key file from BioMart and index probesets to MGI gene symbols

In [23]:
dfX=pd.read_table('../FHS project/Sleep notebook Copy/BioMart_Ensmbl_index/mart_export72_430v2430Av2.txt',index_col=[3])
 
dfX.pop('Affy mouse430 2 probeset') # remove 430V2 probeset info (not needed for 430AV2 indexing)
dfX.head(5)

,Ensembl Gene ID,Description,MGI symbol
Affy mouse430a 2 probeset,,,
1417126_a_at,ENSMUSG00000039221,ribosomal protein L22 like 1 [Source:MGI Symbo...,Rpl22l1
NaN,ENSMUSG00000095611,predicted gene 10597 [Source:MGI Symbol;Acc:MG...,Gm10597
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
NaN,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1


In [24]:
df_Join = df.join(dfX, how='left', sort=True)
df_FINAL1 = df_Join.groupby('MGI symbol').mean()
df_FINAL1[df_FINAL1.index.duplicated()==True]   # checking that no duplicate entries exist in the dataframe

,GSM239868_AK_S_ZT6.CEL,GSM239869_AK_S_ZT6.CEL,GSM239870_AK_S_ZT6.CEL,avg-AK_S_ZT6,log_fold-AK_S_ZT6_vs_AK_SD_ZT6,fold-AK_S_ZT6_vs_AK_SD_ZT6,rawp-AK_S_ZT6_vs_AK_SD_ZT6,adjp-AK_S_ZT6_vs_AK_SD_ZT6,GSM239871_AK_S_ZT12.CEL,GSM239872_AK_S_ZT12.CEL,...,ANOVA-adjp,largest fold,430AV2_ZT6_AK_Enrich,430AV2_ZT6_AK_poolStDev,430AV2_ZT6_AK_Cohens_d,430AV2_ZT6_AK_J,430AV2_ZT6_AK_Hedges_g,430AV2_ZT6_AK_Var_d,430AV2_ZT6_AK_Var_g,430AV2_ZT6_AK_SEg
MGI symbol,,,,,,,,,,,,,,,,,,,,,


### Columns from the list above can then easily be picked to produce files for use later. Examples below given:
 #### df3 = average S and SD expression for the platform and the log-fold changes
 #### df4 = Hedges g  values and associated variance for Meta-analysis (after indexing)

In [25]:
# df3 = df_FINAL1.loc[:,[u'avg-SD', u'avg-S', u'log_fold-S_vs_SD']]
# df3.columns =[prefix+'avg-SD', prefix+'avg-S', prefix+'log_fold-S_vs_SD']
# df3.to_csv('input_files/430AV2_SymbolExpression_forIndex.csv')

In [26]:
df4 = df_FINAL1.loc[:,[u'430AV2_ZT6_AK_Enrich',u'430AV2_ZT6_AK_Hedges_g', u'430AV2_ZT6_AK_Var_g', u'430AV2_ZT6_AK_SEg']]
df4.to_csv('../FHS project/Sleep notebook Copy/IPython_notebooks/input_files/430AV2_ZT6_AK_SymbolforIndexHedges.csv')

In [27]:
df4.head(10)  # check final ouput

,430AV2_ZT6_AK_Enrich,430AV2_ZT6_AK_Hedges_g,430AV2_ZT6_AK_Var_g,430AV2_ZT6_AK_SEg
MGI symbol,,,,
0610005C13Rik,-0.096210,-1.146840,0.591270,0.768941
0610008F07Rik,0.021980,0.126676,0.483004,0.694985
0610009B22Rik,0.039933,1.280942,0.618401,0.786385
0610009D07Rik,-0.008940,-0.074997,0.483093,0.695049
0610009O20Rik,-0.012033,-0.121388,0.482895,0.694906
0610010K14Rik,0.153290,1.164307,0.594634,0.771125
0610012G03Rik,0.113029,0.791394,0.565767,0.749546
0610031J06Rik,0.086633,0.717733,0.524595,0.724289
0610037L13Rik,0.003050,0.030141,0.481742,0.694077
